# Intro
### Goal
1. Open Job Description Page (https://careers.liberty.edu/?job_posting=R000XXXX)<br>
2. Fill in information<br>
3. Paste answers<br>
4. Accept Terms and Conditions<br>
5. Upload resume and recommendation letter<br>
6. Submit<br>
7. Check success message:<br>
        YES = done; NO = record job ID + throw error

### Logic
1. Get the Custom Settings from the 'config.yml' file
2. Start the Browser
3. Complete the Applications

### Citations
https://www.lambdatest.com/blog/how-to-automate-filling-in-web-forms-with-python-using-selenium/

<br><br><br>
# Step 0: Import Necessary Libraries

In [4]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By # CSS selector
from selenium.webdriver.support.ui import WebDriverWait # for wait()
from selenium.webdriver.support import expected_conditions as EC # for wait()
import yaml # read "config.yml" file
import autoit as aut # upload files
import time # wait for upload to finish

# Step 0: Meet My Defined Functions :)

Split screen. Compare 'Free Response' with Job Description

In [5]:
def job_desc_compare(siteID: str):
    # Open Job Desc window
    browser.switch_to.new_window('window')
    browser.get(root_JD + siteID)
    # Get window size
    size = browser.get_window_size()
    fullW = size.get("width")
    fullH = size.get("height")
    # Split right (current window)
    browser.set_window_position(fullW / 2, 0)
    browser.set_window_size(fullW // 2, fullH)
    # Split left
    browser._switch_to.window(browser.window_handles[0]) # Note: windowHandle=left/right not supported for Edge. Must use _switch_to.window(left/right)
    browser.set_window_position(0, 0)
    browser.set_window_size(fullW // 2, fullH)
    waitForever.until(EC.element_to_be_clickable((By.ID, button_page[3]))) # Wait until move on to next page (Employment History)

    # Close Job Desc and switch back
    browser._switch_to.window(browser.window_handles[1])
    browser.close()
    browser._switch_to.window(browser.window_handles[0])
    browser.maximize_window()

Upload file upon request

In [6]:
def uploadFile(fileName: str):
    aut.win_wait_active("Open", 10) # Wait 10 secs for Explorer "Open" window
    aut.control_focus("Open", "Edit1")
    aut.control_set_text("Open", "Edit1", fileName)
    aut.control_click("Open", "Button1")

Pause program

In [7]:
def pause():
    input("Program paused. Press <ENTER> to continue...\n")

<br><br><br>
# STEP I: Get the Custom Settings

Parse the "config.yml" file

In [ ]:
# Read config.yml
with open("config.yml", 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as err:
        print(err)
        print("ERROR: Cannot read \"config.yml\". Mission Aborted :(")

# # Print the resulting 'config' variable
# for key in config.keys():
#     print(key)
#     print(config[key], "\n")

# Import customized responses and settings
driver_path = config["browser"]["driver_path"]
edge_path = config["browser"]["edge_path"]
root = config["job_portal_boilerplate_paths"]["root"] # Website boilerplate Application path
root_JD = config["job_portal_boilerplate_paths"]["root_JD"] # Website boilerplate Job Description path
site_IDs = config["site_ids"]
basic_info = list(config["page_1"].values())
elig_and_major = list(config["page_2"].values())
seven_answers = ["\n".join(response) for response in list(config["page_3"].values())]
docs_to_upload = list(config["docs_to_upload"].values())

Fields and answers

In [10]:
elem_page = [None] * 6 # Holds elements' IDs for 6 pages
ans_page = [None] * 6 # Holds typed answers for 6 pages
button_page = [None] * 6 # Holds buttons' IDs for 6 pages

In [11]:
elem_page[0] = ["input_59_30", "input_59_32", "input_59_63", "input_59_65_1", "input_59_65_2", "input_59_65_3", "input_59_3_1", "input_59_3_3", "input_59_3_4", "input_59_3_5", "input_59_69", "input_59_70", "input_59_4", "input_59_7", "input_59_11"]
ans_page[0] = basic_info
button_page[0] = "gform_next_button_59_18" # Basic info

In [12]:
elem_page[1] = ["input_59_42", "input_59_44", "input_59_45"]
ans_page[1] = elig_and_major
button_page[1] = "gform_next_button_59_41" # Work Study, Major

In [13]:
elem_page[2] = ["input_59_50", "input_59_51", "input_59_52", "input_59_53", "input_59_54", "input_59_55", "input_59_56"]
ans_page[2] = seven_answers
button_page[2] = "gform_next_button_59_23" # Questions

In [14]:
elem_page[3] = ["gform_next_button_59_25"]
button_page[3] = "gform_next_button_59_25" # Skip Employment Histories

In [15]:
elem_page[4] = ["input_59_60", "input_59_61"]
ans_page[4] = ["Yes", "Yes"]
button_page[4] = "gform_next_button_59_28" # Christian Values

In [16]:
elem_page[5] = ["input_59_29_1"] # Read and Agree
button_page[5] = "gform_next_button_59_34"

Files to upload

In [17]:
upload_labels = ["select files, upload resume", "select files, upload letter of recommendation"]
upload_files = docs_to_upload

<br><br><br>
# STEP II: Start the Browser

Necessary info to start Edge browser session

In [22]:
my_service = Service(driver_path)
option = Options()
option.binary_location = edge_path

Start Edge browser session

In [ ]:
global browser
browser = webdriver.Edge(service=my_service, options=option, keep_alive=True)

Set up wait for later

In [ ]:
wait = WebDriverWait(browser, 20) # timeout after 20 secs unresponsive
waitForever = WebDriverWait(browser, 3600)

# Alternative: While content hasn't changed, keep waiting...
# def wait_for_nextPage():
#     srcOld = browser.page_source
#     srcNew = srcOld
#     while srcOld == srcNew:
#         srcNew = browser.page_source

<br><br><br>
# STEP III: Complete the Applications

In [ ]:
if __name__ == "__main__":
    for siteID in site_IDs:
        # 1. Open a website
        browser.get(root + siteID)

        # Full screen
        browser.maximize_window()

        # 2/3/4. Paste Answers
        for i in range(6):
            # Wait for first element to be clickable
            wait.until(EC.element_to_be_clickable((By.ID, elem_page[i][0])))
            
            # Fill out form
            # Page 6 (i == 5) --> Click to check 'Read and Agree"
            if (i == 5): 
                agree_button = browser.find_element(By.ID, elem_page[i][0])
                agree_button.click()
            
            # Page 4 (i == 3) --> Skip Employment History
            
            # Other pages --> Fill out info as normal
            elif (i != 3):
                for elem, ans in zip(elem_page[i], ans_page[i]):
                    browser.find_element(By.ID, elem).send_keys(ans)
            
            # Page 3 (i == 2) --> Stop to revise my questions.
            if i == 2:
                # Split Screen and compare QnA with Job Desc
                job_desc_compare(siteID)
                # Since button is manually clicked by user, skip continue_button.click() below
                continue

            # Go to next page
            continue_button = browser.find_element(By.ID, button_page[i])
            continue_button.click()

        # 5. Upload Files
        # Upload
        for label, file in zip(upload_labels, upload_files):
            label_name = "//button[@aria-label=\"" + label + "\"]"
            # Wait for upload to be ready
            wait.until(EC.element_to_be_clickable((By.XPATH, label_name)))
            # Upload
            upload_button = browser.find_element(By.XPATH, value=label_name)
            upload_button.click()
            uploadFile(file)

            # Wait for upload to finish (Progress Bar appears, then disappears from source code)
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "gpfup__progress-container")))
            waitForever.until(EC.invisibility_of_element_located((By.CLASS_NAME, "gpfup__progress-container")))

        # Go to review page
        time.sleep(1) # Slow down to make sure uploads are in.
        final_button = browser.find_element(By.ID, "gform_next_button_59_71")
        final_button.click()

        # 6. Submit = Manual
        wait.until(EC.element_to_be_clickable((By.ID, "gform_submit_button_59")))
        submit_button = browser.find_element(By.ID, "gform_submit_button_59")
        submit_button.click()

        # 7. Check successful message
        waitForever.until(EC.visibility_of_element_located((By.XPATH, "//h2[text()='You have successfully submitted your application.']")))
        job_title = browser.find_element(By.ID, "job_title_data").text
        print(f"Application #[{siteID}]: \"{job_title}\" all done!")

In [ ]:
pause()